## 5. LangChain 入門（後半）

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

### 5.2. Indexes

In [2]:
import langchain
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.indexes import VectorstoreIndexCreator

loader = DirectoryLoader("./langchain/docs/", glob="**/*.md", loader_cls=UnstructuredMarkdownLoader)
index = VectorstoreIndexCreator().from_loaders([loader])

In [3]:
from langchain.chat_models import ChatOpenAI

langchain.verbose = True

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

result = index.query("LangChainの概要を1文で説明してください。", llm=chat)
print(result)



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
So far, all the chains and agents we've gone through have been stateless. But often, you may want a chain or agent to have some concept of "memory" so that it may remember information about its previous interactions. The clearest and simple example of this is when designing a chatbot - you want it to remember previous messages so it can use context from that to have a better conversation. This would be a type of "short-term memory". On the more complex side, you could imagine a chain/agent remembering key pieces of information over time - this would be a form of "long-term memory". For more concrete ideas on the latter, see this awesome paper.

LangChain provides 

### 5.4. Memory

In [4]:
import langchain
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain

langchain.verbose = True

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

chain = ConversationChain(llm=chat)
result = chain.predict(input="私の名前は岡部です。")
print(result)

chain = ConversationChain(llm=chat)
result = chain.predict(input="私の名前はわかりますか？")
print(result)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 私の名前は岡部です。
AI:

> Finished chain.
こんにちは、岡部さん！お名前を教えてくれてありがとうございます。私はAIです。どのようにお手伝いできますか？


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 私の名前はわかりますか？
AI:

> Finished chain.
はい、あなたの名前はわかりますか？


In [5]:
import langchain
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

langchain.verbose = True

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

memory = ConversationBufferMemory()
chain = ConversationChain(llm=chat, memory=memory)
result = chain.predict(input="私の名前は岡部です。")
print(result)

chain = ConversationChain(llm=chat, memory=memory)
result = chain.predict(input="私の名前はわかりますか？")
print(result)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 私の名前は岡部です。
AI:

> Finished chain.
こんにちは、岡部さん！お名前を教えてくれてありがとうございます。どのようなお手伝いができますか？


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 私の名前は岡部です。
AI: こんにちは、岡部さん！お名前を教えてくれてありがとうございます。どのようなお手伝いができますか？
Human: 私の名前はわかりますか？
AI:

> Finished chain.
はい、先ほどおっしゃった通り、お名前は岡部さんですね。


### 5.5. Agents

In [6]:
import langchain
from langchain.chat_models import ChatOpenAI
from langchain.agents import load_tools
from langchain.agents import initialize_agent

langchain.verbose = True

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

tools = load_tools(["terminal"], llm=chat)
agent_chain = initialize_agent(tools, chat, agent="zero-shot-react-description")

result = agent_chain.run("What is your current directory?")
print(result)



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. You have access to the following tools:

terminal: Run shell commands on this Linux machine.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [terminal]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: What is your current directory?
Thought:

> Finished chain.
I can use the terminal to check the current directory.
Action: terminal
Action Input: pwd
Observation: /home/kakutani/code/openai/udemy/langchain-apps

Thought:

> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as be

/home/kakutani/code/openai/udemy/langchain-apps/env/lib/python3.10/site-packages/langchain/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(



> Finished chain.
I now know the final answer
Final Answer: My current directory is /home/kakutani/code/openai/udemy/langchain-apps

> Finished chain.
My current directory is /home/kakutani/code/openai/udemy/langchain-apps


### 5.8. Chat API に特化した実装

In [7]:
import langchain
import openai
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

openai.log = "debug"
langchain.verbose = True

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
memory = ConversationBufferMemory()

chain = ConversationChain(llm=chat, memory=memory)
result = chain.predict(input="私の名前は岡部です。")
print(result)

chain = ConversationChain(llm=chat, memory=memory)
result = chain.predict(input="私の名前はわかりますか？")
print(result)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 私の名前は岡部です。
AI:


message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
api_version=None data='{"messages": [{"role": "user", "content": "The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\\n\\nCurrent conversation:\\n\\nHuman: \\u79c1\\u306e\\u540d\\u524d\\u306f\\u5ca1\\u90e8\\u3067\\u3059\\u3002\\nAI:"}], "model": "gpt-3.5-turbo", "max_tokens": null, "stream": false, "n": 1, "temperature": 0.0}' message='Post details'
message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=1714 request_id=8f303d0d8c45757414d591db76282ad0 response_code=200
body='{\n  "id": "chatcmpl-7oBsKnPDdpoztw8ocKubqMAqTdOhh",\n  "object": "chat.completion",\n  "created": 1692196404,\n  "model": "gpt-3.5-turbo-0613",\n  "choices": [\n    {\n      "index": 0,\n      "message": {\n   


> Finished chain.
こんにちは、岡部さん！お名前を教えていただきありがとうございます。私はAIです。どのようなお手伝いができますか？


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 私の名前は岡部です。
AI: こんにちは、岡部さん！お名前を教えていただきありがとうございます。私はAIです。どのようなお手伝いができますか？
Human: 私の名前はわかりますか？
AI:

> Finished chain.
はい、先ほどおっしゃった通り、あなたの名前は岡部さんですね。


message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=1572 request_id=4703e2d9a99c75684305563687e8040f response_code=200
body='{\n  "id": "chatcmpl-7oBsMOYpkILkpUWPHVOxoF849tGil",\n  "object": "chat.completion",\n  "created": 1692196406,\n  "model": "gpt-3.5-turbo-0613",\n  "choices": [\n    {\n      "index": 0,\n      "message": {\n        "role": "assistant",\n        "content": "はい、先ほどおっしゃった通り、あなたの名前は岡部さんですね。"\n      },\n      "finish_reason": "stop"\n    }\n  ],\n  "usage": {\n    "prompt_tokens": 133,\n    "completion_tokens": 32,\n    "total_tokens": 165\n  }\n}\n' headers='{\'Date\': \'Wed, 16 Aug 2023 14:33:27 GMT\', \'Content-Type\': \'application/json\', \'Transfer-Encoding\': \'chunked\', \'Connection\': \'keep-alive\', \'access-control-allow-origin\': \'*\', \'Cache-Control\': \'no-cache, must-revalidate\', \'openai-model\': \'gpt-3.5-turbo-0613\', \'openai-organization\': \'user-wx5qxver7o5fbsxy4tt4aja8\', \'openai-processing-ms\': \'1

In [8]:
import langchain
import openai
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

openai.log = "debug"
langchain.verbose = True

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

messages = [
    SystemMessage(content="あなたは優秀なアシスタントです。"),
    HumanMessage(content="私の名前は岡部です。")
]

result = chat(messages)
print(result)

message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
api_version=None data='{"messages": [{"role": "system", "content": "\\u3042\\u306a\\u305f\\u306f\\u512a\\u79c0\\u306a\\u30a2\\u30b7\\u30b9\\u30bf\\u30f3\\u30c8\\u3067\\u3059\\u3002"}, {"role": "user", "content": "\\u79c1\\u306e\\u540d\\u524d\\u306f\\u5ca1\\u90e8\\u3067\\u3059\\u3002"}], "model": "gpt-3.5-turbo", "max_tokens": null, "stream": false, "n": 1, "temperature": 0.0}' message='Post details'


content='はじめまして、岡部さん。どのようにお手伝いできますか？' additional_kwargs={} example=False


message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=980 request_id=952a55bb09770f496ac1937982d8dd4a response_code=200
body='{\n  "id": "chatcmpl-7oBsNNrk2OB8N7w0SxuzEpJmkGGV6",\n  "object": "chat.completion",\n  "created": 1692196407,\n  "model": "gpt-3.5-turbo-0613",\n  "choices": [\n    {\n      "index": 0,\n      "message": {\n        "role": "assistant",\n        "content": "はじめまして、岡部さん。どのようにお手伝いできますか？"\n      },\n      "finish_reason": "stop"\n    }\n  ],\n  "usage": {\n    "prompt_tokens": 38,\n    "completion_tokens": 26,\n    "total_tokens": 64\n  }\n}\n' headers='{\'Date\': \'Wed, 16 Aug 2023 14:33:28 GMT\', \'Content-Type\': \'application/json\', \'Transfer-Encoding\': \'chunked\', \'Connection\': \'keep-alive\', \'access-control-allow-origin\': \'*\', \'Cache-Control\': \'no-cache, must-revalidate\', \'openai-model\': \'gpt-3.5-turbo-0613\', \'openai-organization\': \'user-wx5qxver7o5fbsxy4tt4aja8\', \'openai-processing-ms\': \'980\', 

In [9]:
import langchain
import openai
from langchain.chat_models import ChatOpenAI

openai.log = "debug"
langchain.verbose = True

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
memory = ConversationBufferMemory()

memory.chat_memory.add_user_message("私の名前は岡部です。")
ai_message = chat(memory.chat_memory.messages).content
memory.chat_memory.add_ai_message(ai_message)
print(ai_message)

memory.chat_memory.add_user_message("私の名前はわかりますか？")
ai_message = chat(memory.chat_memory.messages).content
memory.chat_memory.add_ai_message(ai_message)
print(ai_message)

message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
api_version=None data='{"messages": [{"role": "user", "content": "\\u79c1\\u306e\\u540d\\u524d\\u306f\\u5ca1\\u90e8\\u3067\\u3059\\u3002"}], "model": "gpt-3.5-turbo", "max_tokens": null, "stream": false, "n": 1, "temperature": 0.0}' message='Post details'
message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=1073 request_id=19532ded7e2a5bbd9d8b380de8cacf3c response_code=200
body='{\n  "id": "chatcmpl-7oBsP3ef0gHKUxYVWuXyV6irV2Al6",\n  "object": "chat.completion",\n  "created": 1692196409,\n  "model": "gpt-3.5-turbo-0613",\n  "choices": [\n    {\n      "index": 0,\n      "message": {\n        "role": "assistant",\n        "content": "こんにちは、岡部さん。どのようなご用件でしょうか？"\n      },\n      "finish_reason": "stop"\n    }\n  ],\n  "usage": {\n    "prompt_tokens": 17,\n    "completion_tokens": 22,\n    "total_tokens": 39\n  }\n}\n' headers='{\'Date\': \'Wed, 16 Aug 2023 14:3

こんにちは、岡部さん。どのようなご用件でしょうか？
はい、先ほどおっしゃった通り、お名前は岡部さんですね。


message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=1338 request_id=862aa701f046395c463e6ab7a5e90787 response_code=200
body='{\n  "id": "chatcmpl-7oBsQFJOcNalIgyffkfGWPUIxpXR4",\n  "object": "chat.completion",\n  "created": 1692196410,\n  "model": "gpt-3.5-turbo-0613",\n  "choices": [\n    {\n      "index": 0,\n      "message": {\n        "role": "assistant",\n        "content": "はい、先ほどおっしゃった通り、お名前は岡部さんですね。"\n      },\n      "finish_reason": "stop"\n    }\n  ],\n  "usage": {\n    "prompt_tokens": 58,\n    "completion_tokens": 29,\n    "total_tokens": 87\n  }\n}\n' headers='{\'Date\': \'Wed, 16 Aug 2023 14:33:31 GMT\', \'Content-Type\': \'application/json\', \'Transfer-Encoding\': \'chunked\', \'Connection\': \'keep-alive\', \'access-control-allow-origin\': \'*\', \'Cache-Control\': \'no-cache, must-revalidate\', \'openai-model\': \'gpt-3.5-turbo-0613\', \'openai-organization\': \'user-wx5qxver7o5fbsxy4tt4aja8\', \'openai-processing-ms\': \'1338\'